In [3]:
import os
import glob
import rasterio
from rasterio.mask import mask
import geopandas as gpd

In [5]:
def clip_and_save_landsat_bands(input_dir, shapefile_path, output_base_dir):
    """
    Clips Landsat bands based on a shapefile and saves the clipped bands into a single output directory.
    
    Parameters:
    - input_dir: Directory containing folders by year with Landsat band files.
    - shapefile_path: Path to the shapefile used for clipping.
    - output_base_dir: Directory where all clipped images will be saved.
    """
    # Load shapefile and extract geometry
    shapefile = gpd.read_file(shapefile_path)
    shapes = [feature["geometry"] for feature in shapefile.__geo_interface__["features"]]
    
    
    # Iterate through each year folder in input directory
    for year_folder in sorted(os.listdir(input_dir)):
        year_path = os.path.join(input_dir, year_folder)
        if not os.path.isdir(year_path) or not year_folder.isdigit():
            continue  # Skip non-year directories
            
        # Loop through Landsat 42 and Landsat 43 folders if they exist
        for landsat_type in ['Landsat 42', 'Landsat 43']:
            landsat_path = os.path.join(year_path, landsat_type)
            if not os.path.isdir(landsat_path):
                continue
                
                 # Find all band files (e.g., .tif files) in the Landsat folder
            for band_file in glob.glob(os.path.join(landsat_path, "*.tif")):
                try:
                    # Extract band name and create output name based on year, Landsat type, and band
                    with rasterio.open(band_file) as src:
                        clipped_image, clipped_transform = mask(src, shapes, crop=True)
                        
                         # Define output metadata and update with new dimensions
                        out_meta = src.meta.copy()
                        out_meta.update({
                            "driver": "GTiff",
                            "height": clipped_image.shape[1],
                            "width": clipped_image.shape[2],
                            "transform": clipped_transform
                        })
                        
                        # Create output filename based on the original filename
                        band_name = os.path.basename(band_file)
                        output_path = os.path.join(output_base_dir, f"{year_folder}_{landsat_type}_{band_name}")
                        
                            # Ensure output directory exists
                        os.makedirs(output_base_dir, exist_ok=True)
                        
                        # Save the clipped image
                        with rasterio.open(output_path, "w", **out_meta) as dest:
                            dest.write(clipped_image)

                    print(f"Clipped and saved: {output_path}")

                except Exception as e:
                    print(f"Error processing {band_file}: {e}")

# Example Usage
input_directory = "D:/BUET/Thesis/ArcGIS/Radiometric Corrected Images"          # Root directory containing year folders
shapefile = "D:/BUET/Thesis/ArcGIS/study_area/Study_area.shp"                 # Shapefile for clipping
output_directory = "D:/BUET/Thesis/ArcGIS/Clipped Landsat"    # Directory for saving all clipped images

clip_and_save_landsat_bands(input_directory, shapefile, output_directory)

Clipped and saved: D:/BUET/Thesis/ArcGIS/Clipped Landsat\1989_Landsat 42_1989_42_B2.tif
Clipped and saved: D:/BUET/Thesis/ArcGIS/Clipped Landsat\1989_Landsat 42_1989_42_B3.tif
Clipped and saved: D:/BUET/Thesis/ArcGIS/Clipped Landsat\1989_Landsat 42_1989_42_B4.tif
Clipped and saved: D:/BUET/Thesis/ArcGIS/Clipped Landsat\1989_Landsat 43_1989_43_B2.tif
Clipped and saved: D:/BUET/Thesis/ArcGIS/Clipped Landsat\1989_Landsat 43_1989_43_B3.tif
Clipped and saved: D:/BUET/Thesis/ArcGIS/Clipped Landsat\1989_Landsat 43_1989_43_B4.tif
Clipped and saved: D:/BUET/Thesis/ArcGIS/Clipped Landsat\1993_Landsat 42_1993_42_B2.tif
Clipped and saved: D:/BUET/Thesis/ArcGIS/Clipped Landsat\1993_Landsat 42_1993_42_B3.tif
Clipped and saved: D:/BUET/Thesis/ArcGIS/Clipped Landsat\1993_Landsat 42_1993_42_B4.tif
Clipped and saved: D:/BUET/Thesis/ArcGIS/Clipped Landsat\1993_Landsat 43_1993_43_B2.tif
Clipped and saved: D:/BUET/Thesis/ArcGIS/Clipped Landsat\1993_Landsat 43_1993_43_B3.tif
Clipped and saved: D:/BUET/Thesi

Clipped and saved: D:/BUET/Thesis/ArcGIS/Clipped Landsat\2021_Landsat 43_2021_43_B4.tif
Clipped and saved: D:/BUET/Thesis/ArcGIS/Clipped Landsat\2021_Landsat 43_2021_43_B5.tif
Clipped and saved: D:/BUET/Thesis/ArcGIS/Clipped Landsat\2023_Landsat 42_2023_42_B3.tif
Clipped and saved: D:/BUET/Thesis/ArcGIS/Clipped Landsat\2023_Landsat 42_2023_42_B4.tif
Clipped and saved: D:/BUET/Thesis/ArcGIS/Clipped Landsat\2023_Landsat 42_2023_42_B5.tif
Clipped and saved: D:/BUET/Thesis/ArcGIS/Clipped Landsat\2023_Landsat 43_2023_43_B3.tif
Clipped and saved: D:/BUET/Thesis/ArcGIS/Clipped Landsat\2023_Landsat 43_2023_43_B4.tif
Clipped and saved: D:/BUET/Thesis/ArcGIS/Clipped Landsat\2023_Landsat 43_2023_43_B5.tif
